# Tic Tac Toe Coin

To store state of tic tac toe game on Chia blockchain, we create an outer puzzle that wrap the inner tic tac toe puzzle. The coin puzzle outputs the conditions based on the output from the inner tic tac toe puzzle.

## Code
### [coin.clsp](./code/coin.clsp)
```lisp
;   MOD                         : the puzzle itself
;   PLAYER_ONE_INFO             : (PK, PUZZLE_HASH), public key and puzzle hash of the first player
;   PLAYER_TWO_INFO             : (PK, PUZZLE_HASH), public key and puzzle hash of the second player
;   CURRIED_TIC_TAC_TOE_PUZZLE  : tic tac toe puzzle with curried-in current board and next player
;   AMOUNT                      : coin amount (odd to be used with singleton top layer)
;   position                    : next play position as inner solution for curried tic tac toe puzzle

(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
     ...
)     
```

## Generate PKs for player one and two
```sh
cdv inspect keys --random | grep public -i
cdv inspect keys --random | grep public -i
Public Key: ab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0
Public Key: 9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc
```


In [1]:
%%bash
chia version
cdv --version
python --version

1.5.0
cdv, version 1.0.8
Python 3.8.10


In [2]:
# chia libraries
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle)

sys.path.insert(0, "./code")
import tic_tac_toe

# load puzzles
tic_tac_toe_puzzle = load_program("./code/tic-tac-toe.clsp", ["./code", "../../../shared"])
coin_puzzle = load_program("./code/coin.clsp", ["./code", "../../../shared"])

# prepare players info
player_one_pk = bytes.fromhex("ab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0")
player_one_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(player_one_pk).get_tree_hash()
player_two_pk = bytes.fromhex("9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc")
player_two_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(player_two_pk).get_tree_hash()
player_one_info = Program.to([player_one_pk, player_one_hash])
player_two_info = Program.to([player_two_pk, player_two_hash])

amount = 1023
def sim_play(board, player, position):
    # print(f'{board} {player} {position}')

    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle)
    
    conditions = curried_coin_puzzle.run(Program.to([amount, position]))
    coin_puzzle_hash = conditions.at("rrfrf")

    print()
    print_program(conditions)
    print()
    board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
    tic_tac_toe.print_board(board)
    print({
         0 : '',
        -1 : 'tie game',
        120: 'x wins',
        113: 'o wins'
    }[board_state])

    if board_state == 0:
        ## verify puzzle
        player = 'x' if player == 'o' else 'o'
        curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
                Program.to(board), 
                Program.to(player)
            )
        curried_coin_puzzle = coin_puzzle.curry(
            coin_puzzle,
            player_one_info,
            player_two_info,
            curried_tic_tac_toe_puzzle)

        # new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
        assert curried_coin_puzzle.get_tree_hash() == coin_puzzle_hash
        return board, player
    return board, None


## Simulate Winning Board
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until `x` wins.

In [3]:
board, player = sim_play([' '] * 9, 'x', 4)
board, player = sim_play(board, player, 0)
board, player = sim_play(board, player, 3)
board, player = sim_play(board, player, 1)
board, player = sim_play(board, player, 5)



((50 0x9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc 0x24644d734eb6f3bebcc3314494518cf5df9fb8e2614981b45bc753912c85ab19) (73 1023) (51 0x814d67f823d678d36904fc2c65429d270a43261dea5741c243b38d6b785d3909 1023))

   |   |   
---+---+---
   | x |   
---+---+---
   |   |   



((50 0xab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0 0x0c822ff0ca88a532eff683ad7f46b486449a89b513f054c2d9835d22da45dad0) (73 1023) (51 0x0d2b89663c57e838a7c9787f81b3b7a3a46bb3414c58d35387dc58fa307e4a43 1023))

 o |   |   
---+---+---
   | x |   
---+---+---
   |   |   



((50 0x9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc 0x25a89c71e8510e60a26b484eac90002abd93de58d72cb85eafc706965ce0e0fa) (73 1023) (51 0xdf5cb783f6aa2e6ca27a8022d9cf6e12fbadd0830f06da8ff55ec35cabe6b666 1023))

 o |   |   
---+---+---
 x | x |   
---+---+---
   |   |   



((50 0xab7431b52af2ff59d430

## Simulate Tie Game
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until the board is not playable.

In [4]:
board, player = sim_play([' '] * 9, 'x', 0)
board, player = sim_play(board, player, 1)
board, player = sim_play(board, player, 2)
board, player = sim_play(board, player, 4)
board, player = sim_play(board, player, 5)
board, player = sim_play(board, player, 6)
board, player = sim_play(board, player, 7)
board, player = sim_play(board, player, 8)
board, player = sim_play(board, player, 3)


((50 0x9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc 0x78be85f051cd710988b61d7b9176dcbf7f3208c2a5c6a6ffa6688a6c631301f8) (73 1023) (51 0x5c142b04cf5c3e9285a80341642f6330c2067a1e5759409bcb6333fc1bc44dd1 1023))

 x |   |   
---+---+---
   |   |   
---+---+---
   |   |   



((50 0xab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0 0xf0dbb49816f3792bb4952cbdfbf6502207d157d2b575b8b2b5e1de4e8057e56b) (73 1023) (51 0x8f3d6a66454fa6193304fe7ef23c4b6f22e967165b745601d2e085c31fdce71f 1023))

 x | o |   
---+---+---
   |   |   
---+---+---
   |   |   



((50 0x9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc 0x753165bbd1a199b62aa14a65ece45cb03588a1fffb0b4db3b77679fdf23d3f40) (73 1023) (51 0x1d17fb25051d88f0f95919b22e9646b127571d66e8c565ff08ff5d7332d5a4b5 1023))

 x | o | x 
---+---+---
   |   |   
---+---+---
   |   |   



((50 0xab7431b52af2ff59d430